In [1]:
# 读取 pandas 并进行四元数的转换

In [26]:
import pandas as pd
import os

In [27]:
path = "/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/代码_OpenSense/OpenSense_Python/OpenSenseExample/20230527_XsensMVN_NoTorsoFeet_Gait_张德轩"

filename = '20230526_张德轩_Xsens_穿鞋-1.xlsx'
frame_rate = 100

In [28]:
# 切换路径
os.chdir(path)

# 读取 sheet_names
excelfile = pd.ExcelFile(filename)
excelfile.sheet_names

['General Information',
 'Markers',
 'Segment Orientation - Quat',
 'Segment Orientation - Euler',
 'Segment Position',
 'Segment Velocity',
 'Segment Acceleration',
 'Segment Angular Velocity',
 'Segment Angular Acceleration',
 'Joint Angles ZXY',
 'Joint Angles XZY',
 'Ergonomic Joint Angles ZXY',
 'Ergonomic Joint Angles XZY',
 'Center of Mass',
 'Sensor Free Acceleration',
 'Sensor Magnetic Field',
 'Sensor Orientation - Quat',
 'Sensor Orientation - Euler']

In [29]:
# 读取数据
df = pd.read_excel(filename,sheet_name="Sensor Orientation - Quat")
df.columns

Index(['Frame', 'Pelvis q0', 'Pelvis q1', 'Pelvis q2', 'Pelvis q3', 'L5 q0',
       'L5 q1', 'L5 q2', 'L5 q3', 'L3 q0', 'L3 q1', 'L3 q2', 'L3 q3', 'T12 q0',
       'T12 q1', 'T12 q2', 'T12 q3', 'T8 q0', 'T8 q1', 'T8 q2', 'T8 q3',
       'Neck q0', 'Neck q1', 'Neck q2', 'Neck q3', 'Head q0', 'Head q1',
       'Head q2', 'Head q3', 'Right Shoulder q0', 'Right Shoulder q1',
       'Right Shoulder q2', 'Right Shoulder q3', 'Right Upper Arm q0',
       'Right Upper Arm q1', 'Right Upper Arm q2', 'Right Upper Arm q3',
       'Right Forearm q0', 'Right Forearm q1', 'Right Forearm q2',
       'Right Forearm q3', 'Right Hand q0', 'Right Hand q1', 'Right Hand q2',
       'Right Hand q3', 'Left Shoulder q0', 'Left Shoulder q1',
       'Left Shoulder q2', 'Left Shoulder q3', 'Left Upper Arm q0',
       'Left Upper Arm q1', 'Left Upper Arm q2', 'Left Upper Arm q3',
       'Left Forearm q0', 'Left Forearm q1', 'Left Forearm q2',
       'Left Forearm q3', 'Left Hand q0', 'Left Hand q1', 'Left Hand q2

In [30]:
# 定义列表
segments = ["Pelvis","Right Lower Leg","Right Upper Leg",
            "Left Lower Leg","Left Upper Leg"]
quats = [" q0"," q1"," q2"," q3"]
segments_quat = [segment + quat for segment in segments for quat in quats]

In [31]:
import math

# 定义四元数乘法函数
def quaternion_multiply(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2

    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
    z = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2

    return (w, x, y, z)


In [32]:
# 计算表示180度Y轴旋转的四元数
angle = math.pi  # 180度等于π弧度
q_y180 = (math.cos(angle / 2), 0, math.sin(angle / 2), 0)

In [33]:
import numpy as np
segment_quats_result = pd.DataFrame()
time = np.arange(0,len(df)*1/frame_rate,1/frame_rate)
segment_quats_result["time"] = pd.DataFrame(time)
for segment in segments:
    segment_quats = df[[segment + quat for quat in quats]].values

    result = []
    for quat in segment_quats:
        result.append(quaternion_multiply(quat, q_y180))
    result = np.array(result)
    segment_quats_result[[segment + " trans" + quat for quat in quats]] = pd.DataFrame(result)

In [34]:
segment_quats_result.to_csv(os.path.splitext(filename)[0]+".csv",index=False)

# 转换为 opensim 格式

In [35]:
# 设置表头
bodies = ["pelvis","tibia_r","femur_r","tibia_l",'femur_l']
bodies_imu = [body+"_imu" for body in bodies]
print(bodies_imu)
opensim_sto_columns = "time\t"
for body_imu in bodies_imu:
    opensim_sto_columns = opensim_sto_columns + body_imu + "\t"

opensim_sto_columns = opensim_sto_columns.strip() + "\n"

['pelvis_imu', 'tibia_r_imu', 'femur_r_imu', 'tibia_l_imu', 'femur_l_imu']


In [36]:

meatdata = f"""DataRate={frame_rate:.6f}
DataType=Quaternion
version=3
OpenSimVersion=4.4-2022-10-11-798caa8
endheader
"""
header = meatdata + opensim_sto_columns
header

'DataRate=100.000000\nDataType=Quaternion\nversion=3\nOpenSimVersion=4.4-2022-10-11-798caa8\nendheader\ntime\tpelvis_imu\ttibia_r_imu\tfemur_r_imu\ttibia_l_imu\tfemur_l_imu\n'

In [37]:
old_filename = os.path.splitext(filename)[0] + ".csv"
oldfile_path = os.path.join(path,old_filename)

new_filename = os.path.splitext(filename)[0] + ".sto"
newfile_path = os.path.join(path, new_filename)

In [38]:
# 先写入表头内容
with open(newfile_path,"w") as new_f:
    new_f.write(header)

In [39]:
# 写入四元数数据
with open(oldfile_path, "r") as old_f, open(newfile_path, "a") as new_f:
    line_index = 0
    for line in old_f:
        # 去除第一行的表头不进行记录
        line_index += 1
        if line_index > 1:
            new_line = line.split(",")
            # print(new_line)
            str_new = ""
            index = 0
            for i in new_line:
                if index % 4 == 0:
                    str_new = str_new + i.strip() + "\t"
                else:
                    str_new = str_new + i.strip() + ","
                index = index + 1
            new_f.write(str_new.strip()+"\n")

98.33